In [1]:
# installing requirements from txt file
#pip install -r requirements.txt

In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import yfinance as yf
from datetime import datetime
from sklearn.metrics import mean_squared_error

# **Step 1: Data retrieval and indicators calculation**

The idea is to consider a portfolio made only of SPY ETF, as it has been seen that holding an ETF which replicates the Standard and Poor 500 can be one of the best investments you can make.

In [2]:
# downloading monthly prices of the SPY ETF, as VIX data will be monthly and therefore we keep returns as monthly
spy_prices = yf.download('SPY', start = '2005-07-01', end = '2024-12-31', interval = '1mo') # starting since when we have availability for the VIX futures historical term structure
spy_prices = spy_prices['Adj Close']
spy_prices

[*********************100%%**********************]  1 of 1 completed


Date
2005-07-01     85.602951
2005-08-01     84.800423
2005-09-01     85.118668
2005-10-01     83.459297
2005-11-01     87.127510
                 ...    
2024-08-01    560.071289
2024-09-01    570.086792
2024-10-01    566.732605
2024-11-01    600.528809
2024-12-01    584.114075
Name: Adj Close, Length: 234, dtype: float64

In [3]:
spy_rets = spy_prices.pct_change().dropna()
spy_rets

Date
2005-08-01   -0.009375
2005-09-01    0.003753
2005-10-01   -0.019495
2005-11-01    0.043952
2005-12-01   -0.007176
                ...   
2024-08-01    0.023365
2024-09-01    0.017883
2024-10-01   -0.005884
2024-11-01    0.059633
2024-12-01   -0.027334
Name: Adj Close, Length: 233, dtype: float64

Next, we upload VIX future (UX1 index) term structure data downloaded from Bloomberg as of 1/17/2025:

In [4]:
vix_data_ahead = pd.read_excel('VIX_term_structure_20250117.xlsx', header = 0) # uploading VIX futures ahead term structure data
vix_data_ahead = vix_data_ahead.drop(vix_data_ahead.index[0]) # removing first unnecessary row

for i in range(2, len(vix_data_ahead) + 1):
    vix_data_ahead['Period'][i] = datetime.strptime(vix_data_ahead['Period'][i], '%m/%Y')
    
vix_data_ahead

C:\Users\aless\AppData\Local\Temp\ipykernel_11292\2136772644.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vix_data_ahead['Period'][i] = datetime.strptime(vix_data_ahead['Period'][i], '%m/%Y')
C:\Users\aless\AppData\Local\Temp\ipykernel_11292\2136772644.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vix_data_ahead['Period'][i] = datetime.strptime(vix_data_ahead['Period'][i], '%m/%Y')
C:\Users\aless\AppData\Local\Temp\ipykernel_11292\2136772644.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/panda

,Tenor,Ticker,Period,Last Price,Days to expiration
1,Spot,VIX Index,Spot,15.9700,0.0
2,1M,UXF5 Index,2025-01-01 00:00:00,16.1792,30.0
3,1M,UXG5 Index,2025-02-01 00:00:00,17.2382,60.0
4,2M,UXH5 Index,2025-03-01 00:00:00,17.8351,90.0
5,3M,UXJ5 Index,2025-04-01 00:00:00,18.1962,120.0
6,4M,UXK5 Index,2025-05-01 00:00:00,18.4005,150.0
7,5M,UXM5 Index,2025-06-01 00:00:00,18.5484,180.0
8,6M,UXN5 Index,2025-07-01 00:00:00,18.8250,210.0
9,7M,UXQ5 Index,2025-08-01 00:00:00,18.8000,240.0
10,8M,UXU5 Index,2025-09-01 00:00:00,19.1000,270.0


In [6]:
vix_data_hist = pd.read_excel('hist_vix_term_structure.xlsx', header = 0) # uploading VIX futures ahead term structure data
vix_data_hist = vix_data_hist.drop(vix_data_hist.index[0]) # removing first unnecessary row

for i in range(1, len(vix_data_hist) + 1):
    vix_data_hist['Period'][i] = datetime.strptime(vix_data_hist['Period'][i], '%m/%Y')
    
vix_data_hist = vix_data_hist.sort_values(by = 'Period')
vix_data_hist

C:\Users\aless\AppData\Local\Temp\ipykernel_11292\1591811134.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vix_data_hist['Period'][i] = datetime.strptime(vix_data_hist['Period'][i], '%m/%Y')
C:\Users\aless\AppData\Local\Temp\ipykernel_11292\1591811134.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vix_data_hist['Period'][i] = datetime.strptime(vix_data_hist['Period'][i], '%m/%Y')
C:\Users\aless\AppData\Local\Temp\ipykernel_11292\1591811134.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

,Tenor,Ticker,Period,Last Price,Days past
166,5Y,UXG05 Index,2005-02-01 00:00:00,11.4800,7170.0
167,61M,UXH05 Index,2005-03-01 00:00:00,12.2200,7140.0
168,63M,UXK05 Index,2005-05-01 00:00:00,13.1600,7080.0
169,66M,UXQ05 Index,2005-08-01 00:00:00,14.4300,6990.0
162,6Y,UXG06 Index,2006-02-01 00:00:00,12.8700,6810.0
...,...,...,...,...,...
5,292M,UXM4 Index,2024-06-01 00:00:00,16.6821,210.0
6,293M,UXN4 Index,2024-07-01 00:00:00,17.1011,180.0
7,294M,UXQ4 Index,2024-08-01 00:00:00,17.3603,150.0
8,295M,UXU4 Index,2024-09-01 00:00:00,17.6000,120.0


In [7]:
# creating functions for the three indicators which will compose the innovative part of our approach

def rolling_std(series, time_interval): # defining a function for volatility, which we consider as rolling standard deviation
    return series.rolling(window = time_interval).std()

def rolling_correlation(series1, series2, time_interval): # defining a function for the rolling correlation
    return series1.rolling(window = time_interval).corr(series2)

def ROC(series): # defining a function for the Rate Of Change
    return series.pct_change()

Now we are going to define a function that will create a new term structure with newly calculated prices.
The prices will be calculated via linear interpolation with a targeted maturity, so that the new term structure is characterized by constant maturity.

In [8]:
def constant_mat_term_structure(vix_term_structure):
    """
    This function computes the linear interpolation of VIX futures prices 
    for generating a constant maturity term structure.
    
    It takes the VIX dataframe with prices and days to expiration as input,
    and will return the interpolated prices of the VIX futures.
    """
    
    constant_maturity_prices = [] # allocating memory for the prices computed with constant maturity approach
    for i in range(1, len(vix_term_structure)): # looping over all observations of the dataframe fed to the function
        maturity1 = vix_term_structure.loc[i - 1, "Days to expiration"] # first maturity
        maturity2 = vix_term_structure.loc[i, "Days to expiration"] # second maturity
        price1 = vix_term_structure.loc[i - 1, "Last Price"] # first price
        price2 = vix_term_structure.loc[i, "Last Price"] # second price
        
        target_maturity = (maturity1 + maturity2) / 2 # the target maturity is identified as the middle point between maturity 1 and 2
        
        # formula decided to be used for price interpolation
        interpolated_price = price1 * (maturity2 - target_maturity) / (maturity2 - maturity1) + price2 * (target_maturity - maturity1) / (maturity2 - maturity1)
        constant_maturity_prices.append(interpolated_price) # appending each result of the loop in the initially created variable
    
    constant_maturity_prices.insert(0, None) # adding NaN for the first row since it doesn't have a previous contract
    vix_term_structure["Constant Maturity Price"] = constant_maturity_prices # adding the newly computed prices to the original table
    return vix_term_structure

Running the function for our data:

In [9]:
vix_data_ahead = vix_data_ahead.reset_index(drop = True)
constant_maturity_ahead = constant_mat_term_structure(vix_data_ahead) 

vix_data_hist = vix_data_hist.reset_index(drop = True)
vix_data_hist.rename(columns = {'Days past': 'Days to expiration'}, inplace = True)
constant_maturity_hist = constant_mat_term_structure(vix_data_hist) 
vix_data_hist.rename(columns = {'Days to expiration': 'Days past'}, inplace = True)

In [10]:
print("Constant Maturity Term Structure ahead:") # printing the new term structure, made of prices at constant maturity
constant_maturity_ahead[['Period', 'Constant Maturity Price']]

Constant Maturity Term Structure ahead:


,Period,Constant Maturity Price
0,Spot,NaN
1,2025-01-01 00:00:00,16.07460
2,2025-02-01 00:00:00,16.70870
3,2025-03-01 00:00:00,17.53665
4,2025-04-01 00:00:00,18.01565
5,2025-05-01 00:00:00,18.29835
6,2025-06-01 00:00:00,18.47445
7,2025-07-01 00:00:00,18.68670
8,2025-08-01 00:00:00,18.81250
9,2025-09-01 00:00:00,18.95000


In [11]:
print("Constant Maturity Term Structure historical:") # printing the new term structure, made of prices at constant maturity
constant_maturity_hist[['Period', 'Constant Maturity Price']]

Constant Maturity Term Structure historical:


,Period,Constant Maturity Price
0,2005-02-01 00:00:00,NaN
1,2005-03-01 00:00:00,11.85000
2,2005-05-01 00:00:00,12.69000
3,2005-08-01 00:00:00,13.79500
4,2006-02-01 00:00:00,13.65000
...,...,...
164,2024-06-01 00:00:00,16.51625
165,2024-07-01 00:00:00,16.89160
166,2024-08-01 00:00:00,17.23070
167,2024-09-01 00:00:00,17.48015


Calculating the slope of the constant maturity VIX futures term structure in a normalized way, meaning with the difference in prices at the numerator and the difference in days at the denominator:

In [12]:
constant_maturity_ahead['vix_slope'] = constant_maturity_ahead["Constant Maturity Price"].diff() / constant_maturity_ahead["Days to expiration"].diff() # computing the slope of the term structure
constant_maturity_ahead['vix_slope']

0         NaN
1         NaN
2    0.021137
3    0.027598
4    0.015967
5    0.009423
6    0.005870
7    0.007075
8    0.004193
9    0.004583
Name: vix_slope, dtype: float64

In [13]:
constant_maturity_hist["vix_slope"] = constant_maturity_hist["Constant Maturity Price"].diff() / constant_maturity_hist["Days past"].diff() # computing the slope of the term structure
constant_maturity_hist["vix_slope"]

0           NaN
1           NaN
2     -0.014000
3     -0.012278
4      0.000806
         ...   
164   -0.013038
165   -0.012512
166   -0.011303
167   -0.008315
168   -0.052328
Name: vix_slope, Length: 169, dtype: float64

In [14]:
constant_maturity_hist = constant_maturity_hist.set_index('Period', drop = True)
correlation_dataset = constant_maturity_hist.join(spy_rets, how = 'left')
correlation_dataset = correlation_dataset.drop(['Tenor', 'Ticker', 'Last Price', 'Days past', 'Constant Maturity Price'], axis = 1)

Using the previously defined three functions, we are computing the innovative indicators that we are going to use as a double check after the momentum transformer:

In [15]:
vol_indicator = rolling_std(spy_rets, time_interval = 5).dropna() # calculating volatility indicator on the returns of SPY
correlation_indicator = rolling_correlation(correlation_dataset['vix_slope'], correlation_dataset['Adj Close'], time_interval = 5)#.dropna()
roc_indicator = ROC(correlation_dataset['vix_slope']).dropna() # calculating rate of change of the VIX futures constant maturity term structure slope

print(vol_indicator)

Date
2005-12-01    0.024689
2006-01-01    0.026147
2006-02-01    0.026042
2006-03-01    0.020082
2006-04-01    0.013524
                ...   
2024-08-01    0.032965
2024-09-01    0.014218
2024-10-01    0.014054
2024-11-01    0.023751
2024-12-01    0.032741
Name: Adj Close, Length: 229, dtype: float64


In [16]:
print(correlation_indicator)

Period
2005-02-01         NaN
2005-03-01         NaN
2005-05-01         NaN
2005-08-01         NaN
2006-02-01         NaN
                ...   
2024-06-01    0.292605
2024-07-01    0.264485
2024-08-01    0.620178
2024-09-01   -0.911280
2024-10-01    0.866640
Length: 169, dtype: float64


In [17]:
print(roc_indicator)

Period
2005-08-01    -0.123016
2006-02-01    -1.065611
2006-03-01    25.068966
2006-05-01    -1.690476
2006-08-01    -0.022989
                ...    
2024-06-01    -0.420519
2024-07-01    -0.040394
2024-08-01    -0.096577
2024-09-01    -0.264376
2024-10-01     5.293245
Name: vix_slope, Length: 165, dtype: float64


# **Step 2: Training the Momentum Transformer model**

In [18]:
correlation_dataset = correlation_dataset.dropna()
spy_rets_training = correlation_dataset['Adj Close']
vix_slope_training = correlation_dataset['vix_slope']

In [19]:
vol_indicator_training = rolling_std(spy_rets_training, time_interval=5)
correlation_indicator_training = rolling_correlation(spy_rets_training, vix_slope_training, time_interval = 5)
roc_indicator_training = ROC(vix_slope_training)

In [20]:
transformer_train_data = pd.DataFrame({
    'Historical VIX futures slope': vix_slope_training,
    'Stock Returns': spy_rets_training
}).dropna()

transformer_train_data

,Historical VIX futures slope,Stock Returns
Period,,
2005-08-01,-0.012278,-0.009375
2006-02-01,0.000806,0.005726
2006-03-01,0.021000,0.012478
2006-05-01,-0.014500,-0.030120
2006-08-01,-0.014167,0.021823
...,...,...
2024-06-01,-0.013038,0.031951
2024-07-01,-0.012512,0.015374
2024-08-01,-0.011303,0.023365


In [21]:
scaler_transformer = StandardScaler()
scaled_features_transformer = scaler_transformer.fit_transform(transformer_train_data)

X_transformer, y_transformer = [], []
sequence_length = 5

for i in range(sequence_length, len(scaled_features_transformer)):
    X_transformer.append(scaled_features_transformer[i-sequence_length:i])
    y_transformer.append(spy_rets_training.iloc[i])

X_transformer = np.array(X_transformer)
y_transformer = np.array(y_transformer)

In [22]:
X_train_transformer, X_test_transformer, y_train_transformer, y_test_transformer = train_test_split(
    X_transformer, y_transformer, test_size=0.2, random_state=42
)

In [23]:
transformer_model = Sequential([
    LSTM(64, input_shape=(X_transformer.shape[1], X_transformer.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

transformer_model.compile(optimizer='adam', loss='mean_squared_error')
res = transformer_model.fit(X_train_transformer, y_train_transformer, epochs=20, batch_size=16, validation_data=(X_test_transformer, y_test_transformer))

C:\Users\aless\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 153ms/step - loss: 0.2377 - val_loss: 0.2256
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.2105 - val_loss: 0.1920
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1707 - val_loss: 0.1369
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1137 - val_loss: 0.0588
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0403 - val_loss: 0.0088
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0059 - val_loss: 0.0026
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0020 - val_loss: 0.0023
Epoch 10/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 11/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0017 - val_loss: 0.0023
Epoch 12/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0016 - val_loss: 0.0023


In [24]:
forecasts = transformer_model.predict(X_test_transformer) # we can eventually check the forecasts on the testing part of the x variables
rmse = np.sqrt(mean_squared_error(y_test_transformer, forecasts)) # calculating root mean squared error as error measure between testing part of returns and forecasts
print(rmse)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 689ms/step
0.04835584639159406


# **Step 3: Using trained model for forecasting 2025 scenarios**